In [1]:
"""
The ipynb version of the test_yolo script modification.
Now it works with for the iNaturalist directory structure (1 primary folder, n subfolders of different classes)
The output of this script is now a csv file containing the relative filepath and object bounding box of each image
IMPORTANT:
This script is meant to be put in the same directory as the YOLO Keras project which can be downloaded from here:
https://github.com/allanzelener/YAD2K
"""
import argparse
import colorsys
import imghdr
import os
import random

import numpy as np
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageDraw, ImageFont

from yad2k.models.keras_yolo import yolo_eval, yolo_head

C:\Users\Roidi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
# Changeable params
model_path = r'model_data/yolo.h5'
assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'
anchors_path = r'model_data/yolo_anchors.txt'
classes_path = r'model_data/coco_classes.txt'

test_path = r'D:\Resources\Inat_Partial\Aves_Small_SS2_Train\CV_0'
output_path = r'D:\Dummy\temp_out\ss2_cv0_train_bbox.csv'
    
score_threshold = 0.3
iou_threshold = 0.5

In [13]:
# Big block of constant
"""
Custom param, to optimize the object detection functionality of YOLO to detect bird
"""
# When an object is detected as class 1, it is extremely likely to be a bird
# Always return all object instance of class 1 regardless of the number
#['bird', 'aeroplane', 'kite']
CLASS_1 = set([14, 4, 33])
# When an object is detected as class 2, it is likely to be a bird
# In 1 image, when there are more than 1 detections and at least 1 of them belong to class 1, 
# remove all instance of class 2 detection in that scenario
# Otherwise if there are only class 2 detections or worse, the best class 2 instance is returned
# CLASS_2 = ['person', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
#             'giraffe', 'banana', 'apple', 'orange', 'carrot', 'hot dog', 'teddy bear']
CLASS_2 = set([0, 15, 16, 17, 18, 19, 20, 21, 22,
          23, 46, 47, 49, 51, 52, 77])
# If there are only class 3 detections in an image, return the entire image coordinate
# CLASS_3 = ['bicycle', 'car', 'motorbike', 'bus', 'train', 'truck', 'boat', 'traffic light',
#             'fire hydrant', 'stop sign', 'parking meter', 'bench', 'backpack', 'umbrella',
#             'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
#             'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
#             'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'sandwich',
#             'broccoli', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed',
#             'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard',
#             'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
#             'clock', 'vase', 'scissors', 'hair drier', 'toothbrush']

In [4]:
# Part of coide that shouldn't be touched unless necessary
sess = K.get_session()  # TODO: Remove dependence on Tensorflow session.

with open(classes_path) as f:
    class_names = f.readlines()
class_names = [c.strip() for c in class_names]

with open(anchors_path) as f:
    anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    anchors = np.array(anchors).reshape(-1, 2)

yolo_model = load_model(model_path)

# Verify model, anchors, and classes are compatible
num_classes = len(class_names)
num_anchors = len(anchors)
# TODO: Assumes dim ordering is channel last
model_output_channels = yolo_model.layers[-1].output_shape[-1]
assert model_output_channels == num_anchors * (num_classes + 5), \
    'Mismatch between model and given anchor and class sizes. ' \
    'Specify matching anchors and classes with --anchors_path and ' \
    '--classes_path flags.'
print('{} model, anchors, and classes loaded.'.format(model_path))

# Check if model is fully convolutional, assuming channel last order.
model_image_size = yolo_model.layers[0].input_shape[1:3]
is_fixed_size = model_image_size != (None, None)

# Generate colors for drawing bounding boxes.
hsv_tuples = [(x / len(class_names), 1., 1.)
              for x in range(len(class_names))]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(
    map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
        colors))
random.seed(10101)  # Fixed seed for consistent colors across runs.
random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
random.seed(None)  # Reset seed to default.

model_data/yolo.h5 model, anchors, and classes loaded.


C:\Users\Roidi\Anaconda3\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
# Generate output tensor targets for filtered bounding boxes.
# TODO: Wrap these backend operations with Keras layers.
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
input_image_shape = K.placeholder(shape=(2, ))
boxes, scores, classes = yolo_eval(
    yolo_outputs,
    input_image_shape,
    score_threshold=score_threshold,
    iou_threshold=iou_threshold)

In [25]:
# Part that should be changed
for subdir_path in os.listdir(test_path):
    print('Parsing subdir',subdir_path)
    # list of image bounding box string to be saved
    image_bb = []
    for image_file in os.listdir(os.path.join(test_path, subdir_path)):
        cur_image_bb = []
        try:
            image_type = imghdr.what(os.path.join(test_path, subdir_path, image_file))
            if not image_type:
                continue
        except IsADirectoryError:
            continue

        image = Image.open(os.path.join(test_path, subdir_path, image_file))
        if is_fixed_size:  # TODO: When resizing we can use minibatch input.
            resized_image = image.resize(
                tuple(reversed(model_image_size)), Image.BICUBIC)
            image_data = np.array(resized_image, dtype='float32')
        else:
            # Due to skip connection + max pooling in YOLO_v2, inputs must have
            # width and height as multiples of 32.
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            resized_image = image.resize(new_image_size, Image.BICUBIC)
            image_data = np.array(resized_image, dtype='float32')
            print(image_data.shape)

        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = sess.run(
            [boxes, scores, classes],
            feed_dict={
                yolo_model.input: image_data,
                input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        # 3 states of bird existence yes (1), maybe (0), no (-1)
        bird_exist = -1
        if len(set(out_classes) & CLASS_1) > 0:
            bird_exist = 1
        elif len(set(out_classes) & CLASS_2) > 0:
            bird_exist = 0

        for i, c in reversed(list(enumerate(out_classes))):
            # Depending on the state, skip some prediction
            if ((bird_exist == 1 and c not in CLASS_1) or 
                (bird_exist == 0 and c not in CLASS_2) or 
                (bird_exist == -1)):
                continue
            predicted_class = class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            # Add a string containing the subdirectory, image name, 4 bounding box coordinates, and confidence
            cur_image_bb.append(','.join([subdir_path, image_file, 
                                       str(top), str(left), str(bottom), str(right),
                                       str(score)]))
        # If this image has no detected bird, return the bounding box of the whole image
        if len(cur_image_bb) == 0:
            cur_image_bb.append(','.join([subdir_path, image_file, 
                                          '0', '0', 
                                          str(image.size[1]), str(image.size[0]), 
                                          '-1']))

        image_bb += cur_image_bb
    # Now write the bounding box data to the output csv file
    with open(output_path, 'a+', encoding='utf-8') as op:
        for ib in image_bb:
            op.write(ib+'\n')

Parsing subdir Calidris alba
Parsing subdir Gallus gallus domesticus
Parsing subdir Geococcyx californianus
Parsing subdir Phoenicopterus roseus
Parsing subdir Picoides villosus
Parsing subdir Spheniscus demersus
Parsing subdir Sterna striata
Parsing subdir Struthio camelus
Parsing subdir Thryothorus ludovicianus
Parsing subdir Tyrannus verticalis


In [ ]:
sess.close()